<a href="https://colab.research.google.com/github/rahadiyant/Data_AWS_Maritim/blob/main/AWS4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit==1.27.0

In [ ]:
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install scipy
!pip install plotly
!pip install openpyxl

In [ ]:
%%writefile aws4.py
import streamlit as st
from datetime import time, date
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
import numpy as np

sns.set_style("darkgrid")

st.title("Data Cuaca")

st.markdown("""
    Dataset ini berisi informasi tentang parameter cuaca yang dikumpulkan dari stasiun AWS.

    ## Pernyataan Masalah
    Kita ingin menganalisis data cuaca dan membangun model prediktif untuk beberapa parameter cuaca.
### Deskripsi Data
    - Waktu
    - Winspeed
    - Winddir
    - Temp
    - Rh
    - Pressure
    - Rain
    - Solrad
    - Netrad
    - Watertemp
    - Waterlevel
    - Ta_min
    - Ta_max
    - Pancilevel
    - Pancitemp
""")

st.sidebar.title("Operasi pada Dataset")

w1 = st.sidebar.checkbox("Tampilkan tabel", False)
plot = st.sidebar.checkbox("Tampilkan plot", False)
plothist = st.sidebar.checkbox("Tampilkan plot histogram", False)
trainmodel = st.sidebar.checkbox("Latih model", False)
dokfold = st.sidebar.checkbox("Lakukan KFold", False)
distView = st.sidebar.checkbox("Tampilan Distribusi", False)
_3dplot = st.sidebar.checkbox("Plot 3D", False)
linechart = st.sidebar.checkbox("Grafik Garis", False)

@st.cache_data
def read_data():
    return pd.read_excel("https://github.com/rahadiyant/Data_AWS_Maritim/raw/main/1_thn_Data_AWS_Maritim.xlsx")

df = read_data()

if w1:
    st.dataframe(df, width=2000, height=500)

if linechart:
    st.subheader("Grafik Garis")
    numeric_df = df.select_dtypes(include=[np.number])  # Only numerical columns
    st.line_chart(numeric_df)

if plothist:
    st.subheader("Distribusi Setiap Kolom")
    options = df.columns.tolist()
    sel_cols = st.selectbox("Pilih kolom", options, 1)
    st.write(sel_cols)
    fig = go.Figure(data=[go.Histogram(x=df[sel_cols], nbinsx=50)])
    st.plotly_chart(fig)

if plot:
    st.subheader("Korelasi antara Parameter")
    options = df.select_dtypes(include=[np.number]).columns.tolist()  # Only numerical columns
    w7 = st.selectbox("Pilih parameter", options, 1)
    st.write(w7)
    fig, ax = plt.subplots()
    ax.scatter(df[w7], df["rain"])
    ax.set_xlabel(w7)
    ax.set_ylabel("rain")
    ax.set_title(f"{w7} vs rain")
    st.pyplot(fig)

if distView:
    st.subheader("Tampilan Distribusi Gabungan")
    numeric_df = df.select_dtypes(include=[np.number])  # Only numerical columns
    columns_to_plot = st.multiselect('Pilih kolom untuk ditampilkan dalam distribusi', numeric_df.columns.tolist())
    if columns_to_plot:
        hist_data = [numeric_df[col].dropna().values for col in columns_to_plot]
        group_labels = columns_to_plot
        try:
            fig = ff.create_distplot(hist_data, group_labels, bin_size=[.1]*len(hist_data))
            st.plotly_chart(fig)
        except Exception as e:
            st.error(f"Error creating distplot: {e}")

if _3dplot:
    numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numeric_columns) >= 3:
        options = st.multiselect('Pilih kolom untuk plot', numeric_columns, numeric_columns[:3])
        st.write('Anda memilih:', options)
        if len(options) >= 3:
            hist_data = [df[col].dropna().values for col in options]
            trace1 = go.Scatter3d(
                x=hist_data[0],
                y=hist_data[1],
                z=hist_data[2] if len(hist_data) > 2 else hist_data[0],
                mode="markers",
                marker=dict(size=8, colorscale="Viridis"),
            )
            data = [trace1]
            layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0))
            fig = go.Figure(data=data, layout=layout)
            st.plotly_chart(fig)
        else:
            st.error("Pilih setidaknya tiga kolom untuk plot 3D.")
    else:
        st.error("Tidak cukup kolom numerik untuk plot 3D.")

if trainmodel:
    st.header("Pemodelan")
    y = df['rain']
    X = df.drop(columns=['rain']).select_dtypes(include=[np.number]).values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    lrgr = LinearRegression()
    lrgr.fit(X_train, y_train)
    pred = lrgr.predict(X_test)
    mse = mean_squared_error(y_test, pred)
    rmse = sqrt(mse)
    st.markdown(f"""
    Model Regresi Linear telah dilatih :
        - MSE: {mse}
        - RMSE: {rmse}
    """)
    st.success('Model berhasil dilatih')

if dokfold:
    st.subheader("Evaluasi KFold Random Sampling")
    my_bar = st.progress(0)
    X = df.drop(columns=['rain']).select_dtypes(include=[np.number]).values
    y = df['rain'].values
    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    mse_list = []
    rmse_list = []
    r2_list = []
    idx = 1
    fig, ax = plt.subplots()
    for train_index, test_index in kf.split(X):
        my_bar.progress(idx * 10)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        lrgr = LinearRegression()
        lrgr.fit(X_train, y_train)
        pred = lrgr.predict(X_test)
        mse = mean_squared_error(y_test, pred)
        rmse = sqrt(mse)
        r2 = r2_score(y_test, pred)
        mse_list.append(mse)
        rmse_list.append(rmse)
        r2_list.append(r2)
        ax.plot(pred, label=f"dataset-{idx}")
        idx += 1
    ax.legend()
    ax.set_xlabel("Data points")
    ax.set_ylabel("Predictions")

    st.pyplot(fig)
    res = pd.DataFrame({"MSE": mse_list, "RMSE": rmse_list, "r2_SCORE": r2_list})

    st.write(res)
    st.balloons()


In [ ]:
!streamlit run aws4.py & npx localtunnel --port 8501